##Import libraries and data

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [2]:
df1 = pd.read_csv('2019 income data by county.csv', encoding='latin-1', thousands=',')
df2 = pd.read_csv('co-est2019-alldata.csv', encoding='latin-1', thousands=',')
df3 = pd.read_csv('Table_10_Offenses_Known_to_Law_Enforcement_by_State_by_Metropolitan_and_Nonmetropolitan_Counties_2019.csv', thousands=',')

##Data cleaning
Some data cleaning done in Excel, including use of VLookup.

In [3]:
df1 = df1[df1.LineCode == 3]
df1['State1'] = df1['State1'].str.strip()
df1['County'] = df1['County'].str.strip()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##Join data sets

In [4]:
from functools import reduce
data_frames = [df3, df1, df2]
data = reduce(lambda  left,right: pd.merge(left,right,on=['County', 'State1'],
                                            how='outer', validate = 'many_to_many'), data_frames)

In [5]:
data = data.rename(columns={'Violent\ncrime': 'Violent crime'})

##Removing missing data

In [6]:
data = data.dropna(subset=['2019', 'Metro', 'POPESTIMATE2019', 'Violent crime'])

In [24]:
#compression_opts = dict(method='zip',
#                       archive_name='out.csv')  
#data.to_csv('out.zip', index=False,
#          compression=compression_opts)

##Create X and y variables

In [7]:
X = data[['2019', 'POPESTIMATE2019', 'Metro']].values
X = np.asarray(X).astype(np.float32)
y = data['Violent crime'].values

##Split into training and test set

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

##Feature Scaling

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[:, :-1] = sc.fit_transform(X_train[:, :-1])
X_test[:, :-1] = sc.transform(X_test[:, :-1])

##Regression models

###Multiple linear regression

In [29]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

###Support vector regression

In [10]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

###Random forest regression

In [10]:
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)
regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=0)

###XGBoost

In [14]:
from xgboost import XGBRegressor
regressor = XGBRegressor(objective='reg:squarederror')
regressor.fit(X_train, y_train)

XGBRegressor(objective='reg:squarederror')

###Artificial neural network

In [16]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=9, input_dim=3, activation='relu'))
ann.add(tf.keras.layers.Dense(units=9, activation='relu'))
ann.add(tf.keras.layers.Dense(units=9, activation='tanh'))
ann.add(tf.keras.layers.Dense(units=1))
ann.compile(loss='mean_squared_error', optimizer='adam', metrics = ['Accuracy'])

In [17]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 150)

Epoch 1/150
53/53 [==============================] - 1s 4ms/step - loss: 116845.6172 - Accuracy: 0.0601
Epoch 2/150
53/53 [==============================] - 0s 4ms/step - loss: 116768.0547 - Accuracy: 0.0613
Epoch 3/150
53/53 [==============================] - 0s 4ms/step - loss: 116694.0391 - Accuracy: 0.0506
Epoch 4/150
53/53 [==============================] - 0s 4ms/step - loss: 116599.0703 - Accuracy: 0.0321
Epoch 5/150
53/53 [==============================] - 0s 3ms/step - loss: 116498.3594 - Accuracy: 0.0310
Epoch 6/150
53/53 [==============================] - 0s 4ms/step - loss: 116404.8125 - Accuracy: 0.0310
Epoch 7/150
53/53 [==============================] - 0s 4ms/step - loss: 116320.4375 - Accuracy: 0.0310
Epoch 8/150
53/53 [==============================] - 0s 4ms/step - loss: 116237.7344 - Accuracy: 0.0310
Epoch 9/150
53/53 [==============================] - 0s 4ms/step - loss: 116155.6484 - Accuracy: 0.0310
Epoch 10/150
53/53 [==============================] - 0s 3ms/ste

##Predicting test set results

In [31]:
y_pred = regressor.predict(X_test)
np.set_printoptions(precision=2)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[ 3.49e+01  8.40e+01]
 [ 1.09e+01  2.60e+01]
 [ 2.18e+01  1.20e+01]
 [ 3.24e+01  4.40e+01]
 [-1.82e+00  1.00e+01]
 [ 3.42e+01  3.70e+01]
 [ 2.10e+00  2.00e+00]
 [ 2.25e+01  3.30e+01]
 [ 1.58e+01  1.30e+01]
 [ 3.27e+02  8.70e+01]
 [ 4.89e+01  4.80e+01]
 [ 1.04e+02  3.20e+01]
 [ 2.12e+01  6.00e+00]
 [ 1.86e+01  2.00e+01]
 [ 2.08e+00  0.00e+00]
 [ 1.55e+02  1.26e+02]
 [ 1.13e+02  3.90e+01]
 [ 9.96e+01  5.50e+01]
 [ 3.81e+01  2.80e+01]
 [ 1.22e+02  2.92e+02]
 [ 3.22e+01  4.00e+00]
 [ 7.30e+01  1.21e+02]
 [ 1.54e+01  2.00e+01]
 [ 4.91e+00  1.00e+00]
 [ 1.20e+01  1.80e+01]
 [-3.44e+01  1.30e+01]
 [ 8.26e+01  4.00e+00]
 [ 3.11e+01  2.20e+01]
 [ 1.20e+02  5.10e+01]
 [ 2.94e+01  2.30e+01]
 [-1.10e+01  1.50e+01]
 [ 2.29e+02  9.30e+01]
 [ 3.21e+01  4.20e+01]
 [ 4.52e+01  1.30e+01]
 [ 4.80e+01  3.80e+01]
 [ 3.94e+01  4.00e+00]
 [ 3.40e+01  2.50e+01]
 [ 2.86e+01  1.10e+01]
 [ 1.50e+01  1.00e+00]
 [ 4.51e+00  5.00e+00]
 [ 9.04e+00  0.00e+00]
 [ 1.21e+02  2.81e+02]
 [ 8.82e+01  1.19e+02]
 [ 5.52e+02

##Computing R2 value

In [32]:
from sklearn.metrics import r2_score
r2_score(y_test, y_pred)

0.3676861695992486

###Applying K-Fold cross validation

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt

In [34]:
scores = cross_val_score(estimator = regressor, X = X_train, y = y_train, cv = 10, scoring='neg_mean_squared_error')
sqrt(mean(absolute(scores)))

236.65177913564798

In [35]:
print('Variance score: %.2f' % regressor.score(X_test, y_test))

Variance score: 0.37
